#

# Exercise: Web scrape flexcar.gr

Web scrape [flexcar.gr](https://www.google.com/url?q=http://flexcar.gr&sa=D&source=editors&ust=1747567629672978&usg=AOvVaw0IAn-00O3YnivHyDGHvSJo)  
Get the features (brand, model, price, hp, gearbox, extras..) for all leasing car offers.